<a href="https://colab.research.google.com/github/thyungkim/NetDissect-Improve-Accuracy/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import ImageNet
from torchvision import models
from torchvision.datasets import CIFAR100


In [34]:
# Load the Places365 dataset
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

root = os.path.expanduser("~/.cache")
train_data = CIFAR100(root, download=True, train=True, transform = transform)
test_data = CIFAR100(root, download=True, train=False, transform = transform)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
model = models.vgg16(pretrained = False)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
def acc(dataloader, model):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in dataloader:
          images, labels = data
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  accuracy = correct / total
  return accuracy

In [20]:
def train(model, epoch, optimizer, criterion, dataloader):
  for i in range(epoch):
      running_loss = 0.0
      for i, data in enumerate(dataloader, 0):
          inputs, labels = data
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
      print(f'Epoch {epoch + 1} loss: {running_loss / (i + 1)}')

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import models



trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

# Create the VGG-16 model
model = models.vgg16(pretrained=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

train(model, 100, optimizer, criterion, trainloader)
train_accuracy = acc(trainloader, model)
test_accuracy = acc(testloader, model)


In [28]:
trainloader